In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime
import base64

st.set_page_config(page_title="Comprehensive Soldier Health Dashboard", layout="wide")

st.sidebar.title("Settings")
def set_theme_mode():
    theme = st.sidebar.radio("Select Theme Mode", ["Light", "Dark"], index=1)
    return "plotly_dark" if theme == "Dark" else "ggplot2"

theme_mode = set_theme_mode()

st.markdown("""
    <h1 style='text-align: center; font-size: 100px; color: #ffffff;'>COMBAT READINESS</h1>
    <h5 style='text-align: center; color: gray;'>Enhancing Mission Success Through Predictive Analysis</h5>
""", unsafe_allow_html=True)

image_path = "D:\\OMKARS FOLDER\\Combat_Readiness\\CR2.webp"
try:
    st.image(image_path, use_container_width=True, caption="🪖Welcome to the COMBAT READINESS system🪖")
except Exception as e:
    st.write("Please add an appropriate image to display.")

uploaded_file = st.sidebar.file_uploader("Upload Soldier's Health Data", type="csv")

if uploaded_file:
    try:
        data = pd.read_csv(uploaded_file, parse_dates=['Timestamp'])
        required_columns = [
            "Timestamp", "Oxygen Level (%)", "Hydration Level (%)", "Stress Level (%)",
            "Heart Rate (bpm)", "Temperature (°C)", "Systolic (mmHg)", "Diastolic (mmHg)", "Distance Covered (km)"
        ]
        missing_columns = [col for col in required_columns if col not in data.columns]
        if missing_columns:
            st.error(f"Missing columns in the dataset: {', '.join(missing_columns)}")
            st.stop()
        st.sidebar.success("Dataset loaded successfully!")
    except Exception as e:
        st.sidebar.error(f"Error loading dataset: {e}")
        st.stop()
else:
    st.markdown("<h3 style='text-align: center; color: gray;'>Please upload a dataset to get started</h3>", unsafe_allow_html=True)
    st.stop()

tabs = st.tabs([
    "Health Overview", "Health Analysis", "Predictive Analysis", "Task Insights", "Health Recommendations", "Health Report"
])

with tabs[0]:
    st.markdown("""
        <style>
            .metric-card {
                background: linear-gradient(135deg, #0f2027, #203a43, #2c5364);
                padding: 20px;
                border-radius: 15px;
                box-shadow: 0px 0px 15px rgba(0, 255, 255, 0.6);
                text-align: center;
                font-family: 'Arial Black';
                color: white;
            }
        </style>
    """, unsafe_allow_html=True)

    st.markdown("""
        <h1 style='text-align: center; font-size: 50px; color: #00ffff;'>
            🛡️ Soldier Health Overview 🛡️
        </h1>
        <h3 style='text-align: center; color: gray;'>
            Health Monitoring | AI-Powered Insights | Mission Readiness
        </h3>
    """, unsafe_allow_html=True)

    metrics = [
        ("💓 Avg Heart Rate", "Heart Rate (bpm)", "bpm"),
        ("🫁 Avg Oxygen Level", "Oxygen Level (%)", "%"),
        ("⚡ Avg Stress Level", "Stress Level (%)", "%"),
        ("🏃 Total Distance", "Distance Covered (km)", "km"),
        ("💧 Avg Hydration Level", "Hydration Level (%)", "%"),
        ("🌡️ Avg Temperature", "Temperature (°C)", "°C"),
        ("🩸 Systolic BP", "Systolic (mmHg)", "mmHg"),
        ("🩸 Diastolic BP", "Diastolic (mmHg)", "mmHg")
    ]

    cols = st.columns(4)
    for i, (label, key, unit) in enumerate(metrics):
        with cols[i % 4]:
            st.markdown(f"""
                <div class='metric-card'>
                    <h3>{label}</h3>
                    <h2>{data[key].mean():.2f} {unit}</h2>
                </div>
            """, unsafe_allow_html=True)

with tabs[1]:
    st.header("Health Analysis")
    min_time, max_time = pd.to_datetime(data['Timestamp']).min().to_pydatetime(), pd.to_datetime(data['Timestamp']).max().to_pydatetime()
    selected_time = st.slider(
        "Select Time Range:",
        min_value=min_time, max_value=max_time,
        value=(min_time, max_time), format="MM/DD/YY - hh:mm"
    )
    filtered_data = data[(data['Timestamp'] >= selected_time[0]) & (data['Timestamp'] <= selected_time[1])]
    st.subheader("Interactive Visualizations")

    def get_detailed_conclusion(value, thresholds, conclusions):
        for i, threshold in enumerate(thresholds):
            if value <= threshold:
                return conclusions[i]
        return conclusions[-1]

    categories = ["Oxygen Level (%)", "Hydration Level (%)", "Stress Level (%)", "Heart Rate (bpm)", "Temperature (°C)"]
    values = [filtered_data[col].mean() for col in categories]
    values.append(values[0])
    fig_radar = go.Figure()
    fig_radar.add_trace(go.Scatterpolar(r=values, theta=categories + [categories[0]], fill='toself'))
    fig_radar.update_layout(title="Overall Health Metrics Radar Chart")
    st.plotly_chart(fig_radar, use_container_width=True)

    avg_values = [filtered_data[col].mean() for col in ["Oxygen Level (%)", "Hydration Level (%)", "Stress Level (%)", "Heart Rate (bpm)", "Temperature (°C)"]]
    radar_conclusion = get_detailed_conclusion(
        max(avg_values),
        [70, 85, 95],
        ["Overall health is stable, with all key metrics within a normal range. Soldiers appear fit for duty.",
         "Some parameters show minor deviations, requiring occasional monitoring to ensure sustained performance.",
         "Significant fluctuations detected in key health metrics, which may impact operational efficiency and require medical review."]
    )
    st.markdown(f"<div style='background-color: #0d2323; padding: 10px; border-radius: 10px; color: white;'>{radar_conclusion}</div>", unsafe_allow_html=True)

    fig_area = px.area(
        filtered_data, x='Timestamp', y=["Oxygen Level (%)", "Hydration Level (%)", "Stress Level (%)"],
        title="Health Trends Over Time", template=theme_mode
    )
    st.plotly_chart(fig_area, use_container_width=True)

    hydration_trend = filtered_data["Hydration Level (%)"].mean()
    stacked_area_conclusion = get_detailed_conclusion(
        hydration_trend,
        [50, 65, 80],
        ["Severe hydration issues detected, posing a risk of dehydration and fatigue among soldiers.",
         "Hydration levels show minor fluctuations, which may affect endurance over prolonged missions.",
         "Hydration levels remain consistent, ensuring soldiers maintain peak physical performance."]
    )
    st.markdown(f"<div style='background-color: #0d2323; padding: 10px; border-radius: 10px; color: white;'>{stacked_area_conclusion}</div>", unsafe_allow_html=True)

    fig_bubble = px.scatter(
        filtered_data, x="Heart Rate (bpm)", y="Distance Covered (km)",
        size="Stress Level (%)", color="Hydration Level (%)",
        title="Heart Rate vs Distance Covered (Bubble Size: Stress Level)", template=theme_mode
    )
    st.plotly_chart(fig_bubble, use_container_width=True)

    stress_impact = filtered_data["Stress Level (%)"].mean()
    bubble_chart_conclusion = get_detailed_conclusion(
        stress_impact,
        [40, 60, 80],
        ["Stress levels are well-managed, indicating mental resilience and readiness for missions.",
         "Moderate stress detected, which may lead to fatigue if not managed through rest and recovery strategies.",
         "High stress levels detected, requiring immediate intervention to prevent burnout and impaired decision-making."]
    )
    st.markdown(f"<div style='background-color: #0d2323; padding: 10px; border-radius: 10px; color: white;'>{bubble_chart_conclusion}</div>", unsafe_allow_html=True)

    readiness_score = (
        filtered_data["Oxygen Level (%)"].mean() * 0.3 +
        filtered_data["Hydration Level (%)"].mean() * 0.2 +
        (100 - filtered_data["Stress Level (%)"].mean()) * 0.2 +
        (37.0 / filtered_data["Temperature (°C)"].mean()) * 0.15 +
        (80 / filtered_data["Heart Rate (bpm)"].mean()) * 0.15
    )
    readiness_score = max(0, min(100, readiness_score))

    fig_progress = go.Figure(go.Indicator(
        mode="gauge+number",
        value=readiness_score,
        title={"text": "Readiness Score (%)"},
        gauge={"axis": {"range": [0, 100]}, "bar": {"color": "green"}}
    ))
    st.plotly_chart(fig_progress, use_container_width=True)

    readiness_score = (filtered_data["Oxygen Level (%)"].mean() * 0.3 + filtered_data["Hydration Level (%)"].mean() * 0.2 +
        (100 - filtered_data["Stress Level (%)"].mean()) * 0.2 + (37.0 / filtered_data["Temperature (°C)"].mean()) * 0.15 +
        (80 / filtered_data["Heart Rate (bpm)"].mean()) * 0.15)
    readiness_score = max(0, min(100, readiness_score))
    readiness_conclusion = get_detailed_conclusion(
        readiness_score,
        [50, 75, 90],
        ["Readiness is below optimal, indicating that soldiers may struggle with extended operations. Intervention is required.",
         "Moderate readiness detected, but slight improvements in hydration and recovery could enhance performance.",
         "High readiness detected, ensuring that soldiers are well-prepared for mission execution."]
    )
    st.markdown(f"<div style='background-color: #0d2323; padding: 10px; border-radius: 10px; color: white;'>{readiness_conclusion}</div>", unsafe_allow_html=True)

    correlation = filtered_data.select_dtypes(include=["number"]).corr()
    fig_heatmap = px.imshow(correlation, text_auto=True, color_continuous_scale="Viridis", title="Correlation Heatmap")
    st.plotly_chart(fig_heatmap, use_container_width=True)
    correlation = filtered_data.select_dtypes(include=["number"]).corr().abs().mean().mean()
    heatmap_conclusion = get_detailed_conclusion(
        correlation,
        [0.3, 0.6, 0.8],
        ["Weak correlations among health parameters, requiring diverse monitoring for early issue detection.",
         "Moderate correlations detected, indicating some dependencies between hydration, heart rate, and stress levels.",
         "Strong correlations among key health metrics, providing a clear health trend for preventive interventions."]
    )
    st.markdown(f"<div style='background-color: #0d2323; padding: 10px; border-radius: 10px; color: white;'>{heatmap_conclusion}</div>", unsafe_allow_html=True)

    fig_box = px.box(filtered_data, y=["Stress Level (%)", "Hydration Level (%)"], title="Stress & Hydration Level Variability")
    st.plotly_chart(fig_box, use_container_width=True)

    hydration_variability = filtered_data["Hydration Level (%)"].std()
    boxplot_conclusion = get_detailed_conclusion(
        hydration_variability,
        [5, 15, 25],
        ["Hydration levels are stable, indicating good fluid intake among soldiers.",
         "Moderate hydration variability detected, suggesting occasional inconsistencies in water intake.",
         "Significant variation in hydration levels observed, which may affect endurance and performance in extended operations."]
    )
    st.markdown(f"<div style='background-color: #0d2323; padding: 10px; border-radius: 10px; color: white;'>{boxplot_conclusion}</div>", unsafe_allow_html=True)

    fig_hist = px.histogram(filtered_data, x="Heart Rate (bpm)", nbins=20, color_discrete_sequence=['orange'], title="Heart Rate Distribution")
    st.plotly_chart(fig_hist, use_container_width=True)

    heart_rate_std = filtered_data["Heart Rate (bpm)"].std()
    histogram_conclusion = get_detailed_conclusion(
        heart_rate_std,
        [10, 20, 30],
        ["Heart rate remains within a normal range, suggesting good cardiovascular stability.",
         "Mild fluctuations in heart rate detected, which may indicate occasional physical exertion.",
         "Irregular heart rate fluctuations observed, requiring monitoring for potential cardiovascular strain."]
    )
    st.markdown(f"<div style='background-color: #0d2323; padding: 10px; border-radius: 10px; color: white;'>{histogram_conclusion}</div>", unsafe_allow_html=True)

    fig_line = px.line(
        filtered_data, x='Timestamp', y=["Oxygen Level (%)", "Heart Rate (bpm)", "Hydration Level (%)"],
        title="Time-Series Trends of Health Parameters", template=theme_mode
    )
    st.plotly_chart(fig_line, use_container_width=True)

    oxygen_variability = filtered_data["Oxygen Level (%)"].std()
    time_series_conclusion = get_detailed_conclusion(
        oxygen_variability,
        [2, 5, 8],
        ["Oxygen levels remain stable, supporting sustained endurance in training and operations.",
         "Moderate oxygen level fluctuations detected, which could indicate changes in physical exertion levels.",
         "Significant oxygen variability observed, requiring further assessment to prevent performance degradation."]
    )
    st.markdown(f"<div style='background-color: #0d2323; padding: 10px; border-radius: 10px; color: white;'>{time_series_conclusion}</div>", unsafe_allow_html=True)

    fig_violin = px.violin(filtered_data, y="Stress Level (%)", box=True, points="all", title="Distribution of Stress Levels")
    st.plotly_chart(fig_violin, use_container_width=True)

    stress_variability = filtered_data["Stress Level (%)"].std()
    violin_conclusion = get_detailed_conclusion(
        stress_variability,
        [5, 15, 25],
        ["Stress levels are consistently low, ensuring high cognitive and physical efficiency.",
         "Moderate stress variability detected, suggesting fluctuating workloads or environmental conditions.",
         "High variability in stress levels observed, which may lead to fatigue and impaired decision-making."]
    )
    st.markdown(f"<div style='background-color: #0d2323; padding: 10px; border-radius: 10px; color: white;'>{violin_conclusion}</div>", unsafe_allow_html=True)

    hydration_levels = pd.cut(filtered_data["Hydration Level (%)"], bins=[0, 50, 75, 100], labels=["Low", "Moderate", "High"])
    fig_donut = px.pie(names=hydration_levels, title="Hydration Level Distribution", hole=0.4)
    st.plotly_chart(fig_donut, use_container_width=True)

    hydration_levels = pd.cut(filtered_data["Hydration Level (%)"], bins=[0, 50, 75, 100], labels=["Low", "Moderate", "High"])
    high_hydration_count = (hydration_levels == "High").sum()
    donut_conclusion = get_detailed_conclusion(
        high_hydration_count / len(filtered_data),
        [0.3, 0.6, 0.8],
        ["Majority of soldiers exhibit low hydration levels, which could impact stamina and cognitive function.",
         "Moderate hydration levels detected across the unit, with room for improvement in water intake strategies.",
         "Most soldiers are well-hydrated, supporting peak physical and mental performance during operations."]
    )
    st.markdown(f"<div style='background-color: #0d2323; padding: 10px; border-radius: 10px; color: white;'>{donut_conclusion}</div>", unsafe_allow_html=True)

    import pandas as pd
    import plotly.graph_objects as go
    import streamlit as st
    data["Timestamp"] = pd.to_datetime(data["Timestamp"])
    start_time = "2025-01-01 04:00:00"
    end_time = "2025-01-01 08:59:00"
    filtered_data = data[(data["Timestamp"] >= start_time) & (data["Timestamp"] <= end_time)]
    if filtered_data.empty:
        st.warning("No data available between 4:00 AM and 8:59 AM.")
    else:
        filtered_data["Time_String"] = filtered_data["Timestamp"].dt.strftime('%H:%M')
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=[filtered_data.iloc[0]["Timestamp"]],
            y=[filtered_data.iloc[0]["Oxygen Level (%)"]],
            mode="lines+markers",
            line=dict(width=2),
            name="Oxygen Level (%)"
        ))
        fig.add_trace(go.Scatter(
            x=[filtered_data.iloc[0]["Timestamp"]],
            y=[filtered_data.iloc[0]["Heart Rate (bpm)"]],
            mode="lines+markers",
            line=dict(width=2),
            name="Heart Rate (bpm)"
        ))
        fig.add_trace(go.Scatter(
            x=[filtered_data.iloc[0]["Timestamp"]],
            y=[filtered_data.iloc[0]["Hydration Level (%)"]],
            mode="lines+markers",
            line=dict(width=2),
            name="Hydration Level (%)"
        ))
        frames = []
        for i in range(1, len(filtered_data)):
            frame = go.Frame(
                data=[
                    go.Scatter(
                        x=filtered_data["Timestamp"][:i],
                        y=filtered_data["Oxygen Level (%)"][:i],
                        mode="lines",
                        line=dict(color="#78baec", width=2),
                        name="Oxygen Level (%)"
                    ),
                    go.Scatter(
                        x=filtered_data["Timestamp"][:i],
                        y=filtered_data["Heart Rate (bpm)"][:i],
                        mode="lines",
                        line=dict(color="#f01e2c", width=2),
                        name="Heart Rate (bpm)"
                    ),
                    go.Scatter(
                        x=filtered_data["Timestamp"][:i],
                        y=filtered_data["Hydration Level (%)"][:i],
                        mode="lines",
                        line=dict(color="#0000ff", width=2),
                        name="Hydration Level (%)"
                    ),
                ],
                name=str(i)
            )
            frames.append(frame)
        fig.update(frames=frames)
        fig.update_layout(
            title="Health Parameters Over Time (Animated Line Chart)",
            xaxis=dict(title="Time", type="date", tickformat="%H:%M"),
            yaxis=dict(title="Health Parameters"),
            updatemenus=[{
                "buttons": [
                    {
                        "args": [None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True}],
                        "label": "Play",
                        "method": "animate"
                    },
                    {
                        "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                        "label": "Pause",
                        "method": "animate"
                    }
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 87},
                "showactive": False,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "right",
                "y": 0,
                "yanchor": "top"
            }]
        )
        st.plotly_chart(fig, use_container_width=True)

import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

with tabs[2]:
    st.header("Predictive Modeling")
    X = data[["Oxygen Level (%)", "Hydration Level (%)", "Stress Level (%)", "Heart Rate (bpm)", "Temperature (°C)", "Systolic (mmHg)", "Diastolic (mmHg)"]]
    y = data["Distance Covered (km)"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    model_accuracy = 100 - np.sqrt(mean_squared_error(y_test, y_pred))
    readiness_score = (
        data["Oxygen Level (%)"].mean() * 0.3 +
        data["Hydration Level (%)"].mean() * 0.2 +
        (100 - data["Stress Level (%)"].mean()) * 0.2 +
        (37.0 / data["Temperature (°C)"].mean()) * 0.15 +
        (80 / data["Heart Rate (bpm)"].mean()) * 0.15
    )
    readiness_score = max(0, min(100, readiness_score))
    fig_gauge = go.Figure(go.Indicator(
        mode="gauge+number",
        value=readiness_score,
        title={"text": "Readiness Score (%)", "font": {"size": 24, "color": "white"}},
        gauge={
            "axis": {"range": [0, 100], "tickwidth": 3, "tickcolor": "#d4af37"},
            "bar": {"color": "#00FF00"},
            "steps": [
                {"range": [0, 50], "color": "#8B0000"},
                {"range": [50, 75], "color": "#FFD700"},
                {"range": [75, 100], "color": "#006400"}
            ],
            "borderwidth": 4,
            "bordercolor": "#d4af37"
        },
        number={"font": {"size": 40, "color": "white"}}
    ))
    fig_gauge.update_layout(
        paper_bgcolor="#0d2323",
        font={"color": "white", "family": "Arial Black"}
    )
    st.plotly_chart(fig_gauge, use_container_width=True)
    st.markdown(f"""
        <div style="border-radius: 15px; padding: 15px; background-color: #1e2a38; box-shadow: 0px 0px 15px #00ffcc;">
            <h2 style="color: #00ffcc; text-align: center;">🛡️ Soldier Readiness & Deployment Decision 🛡️</h2>

        <div style="border-left: 5px solid red; padding: 10px; background-color: #660000;">
            <h3 style="color: #ff6666;">🔴 (0 - 50%) - High Risk ⚠️</h3>
            <p style="color: white;">❌ <b>The soldier is in a critical condition and unfit for deployment. Immediate medical attention is required to prevent health deterioration. Deploying them in high-risk missions can compromise both their safety and the mission's success. Commanders must prioritize recovery before considering reassignment.</b></p>
        </div>

        <div style="border-left: 5px solid yellow; padding: 10px; background-color: #665500;">
            <h3 style="color: #ffd700;">🟡 (50 - 75%) - Moderate Readiness ⚠️</h3>
            <p style="color: white;">⚠️ <b>The soldier shows signs of fatigue or mild health issues, requiring close monitoring. Deployment is possible for low-risk missions but not advisable for high-intensity operations. Commanders should ensure rest and recovery before assigning prolonged tasks.</b></p>
        </div>

        <div style="border-left: 5px solid green; padding: 10px; background-color: #004d00;">
            <h3 style="color: #00ff00;">🟢 (75 - 100%) - Mission Ready ✅</h3>
            <p style="color: white;">✅ <b>The soldier is in optimal health, fully fit for deployment, and capable of handling high-risk missions. Endurance, stability, and physical resilience are at peak levels. Commanders can confidently assign them to critical operations requiring sustained performance.</b>.</p>
        </div>

        <p style="color: #00ffcc; font-size: 20px; text-align: center;"><b>📌</b> This <b>AI-driven predictive analysis</b> ensures that <b>only fit soldiers are deployed</b>, enhancing <b>mission success rates</b> and <b>reducing health risks</b>. 🚀💪</p>
    """, unsafe_allow_html=True)
    st.markdown(f"""
        <div style="border-radius: 5px; padding: 5px; background-color: #1e2a38; box-shadow: 0px 0px 5px #00ffcc; text-align: center;">
            <h5 style="color: #00ffcc;">✅ Model Accuracy: {model_accuracy:.2f}%</h5>
        </div>
    """, unsafe_allow_html=True)

with tabs[3]:
    st.header("Task Insights")
    task_categories = []
    for hr in filtered_data["Heart Rate (bpm)"]:
        if hr > 120:
            task_categories.append("Intense Task")
        elif 70 <= hr <= 120:
            task_categories.append("Moderate Task")
        else:
            task_categories.append("Simple Task")
    filtered_data["Task Category"] = task_categories
    fig_task = px.bar(
        filtered_data, x="Timestamp", y="Heart Rate (bpm)", color="Task Category",
        title="Task Intensity Over Time", template=theme_mode,
        color_discrete_map={"Intense Task": "red", "Moderate Task": "yellow", "Simple Task": "green"}
    )
    st.plotly_chart(fig_task, use_container_width=True)
    readiness_score = (
        filtered_data["Oxygen Level (%)"].mean() * 0.3 +
        filtered_data["Hydration Level (%)"].mean() * 0.2 +
        (100 - filtered_data["Stress Level (%)"].mean()) * 0.2 +
        (37.0 / filtered_data["Temperature (°C)"].mean()) * 0.15 +
        (80 / filtered_data["Heart Rate (bpm)"].mean()) * 0.15
    )
    readiness_score = max(0, min(100, readiness_score))
    st.markdown("<h2 style='background-color: #0d2323; color: white; padding: 10px; border-radius: 10px; text-align: center;'>Overall Task Conclusion</h2>", unsafe_allow_html=True)
    intense_count = task_categories.count("Intense Task")
    moderate_count = task_categories.count("Moderate Task")
    simple_count = task_categories.count("Simple Task")
    if readiness_score > 75:
        conclusion = "The soldier is highly prepared and demonstrates strong endurance, handling intense tasks efficiently. Readiness for high-intensity operations is excellent."
        color = "green"
    elif 50 <= readiness_score <= 75:
        conclusion = "The soldier maintains a balanced performance, handling both moderate and intense tasks well. Additional endurance training can further enhance performance."
        color = "#665500"
    else:
        conclusion = "The soldier may require additional endurance training as current readiness is moderate. Performance in high-intensity tasks may need improvement."
        color = "red"
    st.markdown(f"<div style='background-color: {color}; padding: 20px; border-radius: 10px; text-align: center; color: white; font-size: 18px;'>{conclusion}</div>", unsafe_allow_html=True)

with tabs[4]:
    st.markdown("""
        <style>
            .recommendation-card {
                background: linear-gradient(135deg, #0f2027, #203a43, #2c5364);
                padding: 20px;
                border-radius: 15px;
                box-shadow: 0px 0px 15px rgba(0, 255, 255, 0.6);
                text-align: left;
                font-family: 'Arial Black';
                color: white;
            }
            .status-box {
                padding: 15px;
                border-radius: 10px;
                text-align: center;
                font-size: 18px;
                font-weight: bold;
                color: white;
                background-color: {color};
            }
        </style>
    """, unsafe_allow_html=True)
    readiness_score = (
        filtered_data["Oxygen Level (%)"].mean() * 0.3 +
        filtered_data["Hydration Level (%)"].mean() * 0.2 +
        (100 - filtered_data["Stress Level (%)"].mean()) * 0.2 +
        (37.0 / filtered_data["Temperature (°C)"].mean()) * 0.15 +
        (80 / filtered_data["Heart Rate (bpm)"].mean()) * 0.15
    )
    readiness_score = max(0, min(100, readiness_score))
    def get_health_recommendations(readiness_score):
        if readiness_score <= 20.9:
            return ("Critical Condition 🚨", "#8B0000", """
                ⭕Health Status: Extremely high risk, unfit for any deployment. Immediate medical attention required.
                ⭕Heart Rate & BP: Continuous monitoring; stabilize before any movement.
                ⭕Oxygen Levels:Oxygen therapy if saturation drops below normal.
                ⭕Fatigue & Recovery:Strict bed rest, no physical activity allowed.
                ⭕Hydration & Nutrition:High-protein, iron, and vitamin-rich diet with IV fluids if necessary.
                ⭕Mental Readiness:Psychological counseling and stress-relief therapies.
                ⭕Deployment Readiness:Not deployable. Full recovery required before reassessment.
                """)
        elif 21 <= readiness_score <= 40.9:
            return ("High Risk 🚨", "#FF4500", """
                ⭕Health Status:Severe fatigue or health concerns; not fit for critical operations.
                ⭕Heart Rate & BP:Regular monitoring, mild physical activity allowed.
                ⭕Oxygen Levels:Deep breathing exercises and oxygen therapy if needed.
                ⭕Fatigue & Recovery:Limited movement, light stretching allowed under supervision.
                ⭕Hydration & Nutrition:Increased water intake, electrolyte supplements, and a high-energy diet.
                ⭕Mental Readiness:Relaxation exercises, meditation, and sleep regulation.
                ⭕Deployment Readiness:Not deployable. Requires medical attention and monitored rest.
                """)
        elif 41 <= readiness_score <= 60.9:
            return ("Moderate Readiness 🟢", "#FFD700", """
                🟢Health Status:Fatigue detected; can handle light duties but needs recovery time.
                🟢Heart Rate & BP:Slightly unstable; needs periodic checks.
                🟢Oxygen Levels:Normal but should do breathing exercises for endurance.
                🟢Fatigue & Recovery:Engage in low-intensity training with sufficient breaks.
                🟢Hydration & Nutrition:Balanced meals with high-protein and fiber-rich foods.
                🟢Mental Readiness:Cognitive training and mindfulness exercises.
                🟢Deployment Readiness:Limited deployment possible. Assign only low-risk tasks with regular health checks.
                """)
        elif 61 <= readiness_score <= 80.9:
            return ("Good Readiness ✅", "#32CD32", """
                🟢Health Status:Fit for standard duties but not extreme endurance tasks.
                🟢Heart Rate & BP:Stable but should maintain fitness routines.
                🟢Oxygen Levels:Good but needs periodic endurance training.
                🟢Fatigue & Recovery:Minor fatigue present; must have adequate rest cycles.
                🟢Hydration & Nutrition:High-energy meals with protein and complex carbs.
                🟢Mental Readiness:Stress management and focus-building drills.
                🟢Deployment Readiness:Deployable for medium-risk operations. Avoid long-duration, high-intensity missions.
                """)
        else:
            return ("Mission Ready ✅", "#00FF00", """
                ✅Health Status:Fully fit, at peak performance, ready for any mission.
                ✅Heart Rate & BP:Optimal; maintain with regular cardiovascular training.
                ✅Oxygen Levels:High endurance; continue intensive physical training.
                ✅Fatigue & Recovery:Standard training with proper recovery cycles.
                ✅Hydration & Nutrition:Well-balanced diet; maintain hydration and micronutrient intake.
                ✅Mental Readiness:High-stress scenario drills and strategic decision-making training.
                ✅Deployment Readiness:Fully deployable. Ready for high-intensity and long-duration missions.
                """)
    status, color, recommendation = get_health_recommendations(readiness_score)
    st.markdown("""
        <h1 style='text-align: center; font-size: 50px; color: #00ffff;'>🛡️ Health Recommendations 🛡️</h1>
        <h3 style='text-align: center; color: gray;'>Personalized Recovery & Readiness Insights</h3>
    """, unsafe_allow_html=True)
    st.markdown(f"""
        <div class='recommendation-card'>
            <h2>Health Status: {status}</h2>
            <p style='text-align: left; white-space: pre-wrap;'>{recommendation}</p>
        </div>
    """, unsafe_allow_html=True)

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image as RLImage
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from PIL import Image
import io

with tabs[5]:
    st.markdown("""
        <h1 style='text-align: center; font-size: 50px; color: #00ffff;'>📄 Soldier Health Report 📄</h1>
        <h3 style='text-align: center; color: gray;'>Comprehensive Health and Readiness Analysis</h3>
    """, unsafe_allow_html=True)
    readiness_score = (
        filtered_data["Oxygen Level (%)"].mean() * 0.3 +
        filtered_data["Hydration Level (%)"].mean() * 0.2 +
        (100 - filtered_data["Stress Level (%)"].mean()) * 0.2 +
        (37.0 / filtered_data["Temperature (°C)"].mean()) * 0.15 +
        (80 / filtered_data["Heart Rate (bpm)"].mean()) * 0.15
    )
    readiness_score = max(0, min(100, readiness_score))
    def get_health_recommendations(readiness_score):
        if readiness_score <= 20.9:
            return ("Critical Condition 🚨", "#8B0000", """
                ⭕Health Status: Extremely high risk, unfit for any deployment. Immediate medical attention required.
                ⭕Heart Rate & BP: Continuous monitoring; stabilize before any movement.
                ⭕Oxygen Levels: Oxygen therapy if saturation drops below normal.
                ⭕Fatigue & Recovery: Strict bed rest, no physical activity allowed.
                ⭕Hydration & Nutrition: High-protein, iron, and vitamin-rich diet with IV fluids if necessary.
                ⭕Mental Readiness: Psychological counseling and stress-relief therapies.
                ⭕Deployment Readiness: Not deployable. Full recovery required before reassessment.
            """)
        elif 21 <= readiness_score <= 40.9:
            return ("High Risk 🚨", "#FF4500", """
                ⭕Health Status: Severe fatigue or health concerns; not fit for critical operations.
                ⭕Heart Rate & BP: Regular monitoring, mild physical activity allowed.
                ⭕Oxygen Levels: Deep breathing exercises and oxygen therapy if needed.
                ⭕Fatigue & Recovery: Limited movement, light stretching allowed under supervision.
                ⭕Hydration & Nutrition: Increased water intake, electrolyte supplements, and a high-energy diet.
                ⭕Mental Readiness: Relaxation exercises, meditation, and sleep regulation.
                ⭕Deployment Readiness: Not deployable. Requires medical attention and monitored rest.
            """)
        elif 41 <= readiness_score <= 60.9:
            return ("Moderate Readiness 🟢", "#FFD700", """
                🟢Health Status: Fatigue detected; can handle light duties but needs recovery time.
                🟢Heart Rate & BP: Slightly unstable; needs periodic checks.
                🟢Oxygen Levels: Normal but should do breathing exercises for endurance.
                🟢Fatigue & Recovery: Engage in low-intensity training with sufficient breaks.
                🟢Hydration & Nutrition: Balanced meals with high-protein and fiber-rich foods.
                🟢Mental Readiness: Cognitive training and mindfulness exercises.
                🟢Deployment Readiness: Limited deployment possible. Assign only low-risk tasks with regular health checks.
            """)
        elif 61 <= readiness_score <= 80.9:
            return ("Good Readiness ✅", "#32CD32", """
                🟢Health Status: Fit for standard duties but not extreme endurance tasks.
                🟢Heart Rate & BP: Stable but should maintain fitness routines.
                🟢Oxygen Levels: Good but needs periodic endurance training.
                🟢Fatigue & Recovery: Minor fatigue present; must have adequate rest cycles.
                🟢Hydration & Nutrition: High-energy meals with protein and complex carbs.
                🟢Mental Readiness: Stress management and focus-building drills.
                🟢Deployment Readiness: Deployable for medium-risk operations. Avoid long-duration, high-intensity missions.
            """)
        else:
            return ("Mission Ready ✅", "#00FF00", """
                ✅Health Status: Fully fit, at peak performance, ready for any mission.
                ✅Heart Rate & BP: Optimal; maintain with regular cardiovascular training.
                ✅Oxygen Levels: High endurance; continue intensive physical training.
                ✅Fatigue & Recovery: Standard training with proper recovery cycles.
                ✅Hydration & Nutrition: Well-balanced diet; maintain hydration and micronutrient intake.
                ✅Mental Readiness: High-stress scenario drills and strategic decision-making training.
                ✅Deployment Readiness: Fully deployable. Ready for high-intensity and long-duration missions.
            """)
    status, rec_color, recommendation = get_health_recommendations(readiness_score)
    st.markdown("""
        <h1 style='text-align: center; font-size: 50px; color: #00ffff;'>📄 Soldier Health Report 📄</h1>
        <h3 style='text-align: center; color: gray;'>Comprehensive Health and Readiness Analysis</h3>
    """, unsafe_allow_html=True)
    dashboard_metrics = [
        ["Metric", "Value"],
        ["Avg Heart Rate", f"{data['Heart Rate (bpm)'].mean():.2f} bpm"],
        ["Avg Oxygen Level", f"{data['Oxygen Level (%)'].mean():.2f} %"],
        ["Avg Stress Level", f"{data['Stress Level (%)'].mean():.2f} %"],
        ["Total Distance", f"{data['Distance Covered (km)'].sum():.2f} km"],
        ["Avg Hydration Level", f"{data['Hydration Level (%)'].mean():.2f} %"],
        ["Avg Temperature", f"{data['Temperature (°C)'].mean():.2f} °C"],
        ["Systolic BP", f"{data['Systolic (mmHg)'].mean():.2f} mmHg"],
        ["Diastolic BP", f"{data['Diastolic (mmHg)'].mean():.2f} mmHg"]
    ]
    task_categories = []
    for hr in filtered_data["Heart Rate (bpm)"]:
        if hr > 120:
            task_categories.append("Intense Task")
        elif 70 <= hr <= 120:
            task_categories.append("Moderate Task")
        else:
            task_categories.append("Simple Task")
    filtered_data["Task Category"] = task_categories
    intense_count = task_categories.count("Intense Task")
    moderate_count = task_categories.count("Moderate Task")
    simple_count = task_categories.count("Simple Task")
    task_summary = [
        ["Task Category", "Count"],
        ["Intense Task", intense_count],
        ["Moderate Task", moderate_count],
        ["Simple Task", simple_count]
    ]
    def generate_pdf_report():
        buffer = io.BytesIO()
        doc = SimpleDocTemplate(buffer, pagesize=letter, rightMargin=40, leftMargin=40, topMargin=40, bottomMargin=40)
        styles = getSampleStyleSheet()
        styleH = styles['Heading1']
        styleN = styles['Normal']
        flowables = []
        flowables.append(Paragraph("Soldier Health Report", styleH))
        flowables.append(Spacer(1, 12))
        flowables.append(Paragraph(f"Readiness Score: {readiness_score:.2f}%", styleN))
        flowables.append(Paragraph(f"Health Status: {status}", styleN))
        flowables.append(Spacer(1, 12))
        flowables.append(Paragraph("Dashboard Overview", styleH))
        t = Table(dashboard_metrics)
        t.setStyle(TableStyle([
            ('BACKGROUND', (0,0), (-1,0), colors.grey),
            ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
            ('ALIGN', (0,0), (-1,-1), 'CENTER'),
            ('GRID', (0,0), (-1,-1), 1, colors.black)
        ]))
        flowables.append(t)
        flowables.append(Spacer(1, 12))
        flowables.append(Paragraph("Gauge Meter", styleN))
        flowables.append(RLImage(gauge_img_path, width=400, height=200))
        flowables.append(Spacer(1, 6))
        flowables.append(Paragraph("Conclusion: The gauge meter provides an at-a-glance view of the overall readiness score.", styleN))
        flowables.append(Spacer(1, 12))
        flowables.append(Paragraph("Overall Health Metrics Radar Chart", styleN))
        flowables.append(RLImage(radar_img_path, width=400, height=300))
        flowables.append(Spacer(1, 6))
        flowables.append(Paragraph("Conclusion: The radar chart displays the balance across key health metrics, indicating strengths and areas for improvement.", styleN))
        flowables.append(Spacer(1, 12))
        flowables.append(Paragraph("Heart Rate Over Time", styleN))
        flowables.append(RLImage(hr_img_path, width=400, height=300))
        flowables.append(Spacer(1, 6))
        flowables.append(Paragraph("Conclusion: The heart rate trend shows fluctuations over time, which aids in cardiovascular assessment.", styleN))
        flowables.append(Spacer(1, 12))
        flowables.append(Paragraph("Oxygen Level Over Time", styleN))
        flowables.append(RLImage(oxygen_img_path, width=400, height=300))
        flowables.append(Spacer(1, 6))
        flowables.append(Paragraph("Conclusion: The oxygen level trend indicates respiratory efficiency and highlights potential oxygen saturation issues.", styleN))
        flowables.append(Spacer(1, 12))
        flowables.append(Paragraph("Stress Level Over Time", styleN))
        flowables.append(RLImage(stress_img_path, width=400, height=300))
        flowables.append(Spacer(1, 6))
        flowables.append(Paragraph("Conclusion: The stress level trend identifies variations in stress that may impact overall performance.", styleN))
        flowables.append(Spacer(1, 12))
        flowables.append(Paragraph("Hydration Level Over Time", styleN))
        flowables.append(RLImage(hydration_img_path, width=400, height=300))
        flowables.append(Spacer(1, 6))
        flowables.append(Paragraph("Conclusion: The hydration level trend offers insights into fluid intake consistency and possible dehydration risks.", styleN))
        flowables.append(Spacer(1, 12))
        flowables.append(Paragraph("Advanced Health Recommendations", styleH))
        flowables.append(Paragraph(recommendation.replace('\n', '<br />'), styleN))
        flowables.append(Spacer(1, 12))
        flowables.append(Paragraph("Task Insights Summary", styleH))
        t2 = Table(task_summary)
        t2.setStyle(TableStyle([
            ('BACKGROUND', (0,0), (-1,0), colors.grey),
            ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
            ('ALIGN', (0,0), (-1,-1), 'CENTER'),
            ('GRID', (0,0), (-1,-1), 1, colors.black)
        ]))
        flowables.append(t2)
        flowables.append(Spacer(1, 12))
        doc.build(flowables)
        buffer.seek(0)
        return buffer
    pdf_buffer = generate_pdf_report()
    st.download_button(label="📥 Download Detailed Health Report",
                       data=pdf_buffer,
                       file_name="Soldier_Health_Report.pdf",
                       mime="application/pdf")
